In [1]:
__author__ = 'Anna Smelova'

### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

!!!В первую очередь делаем сайт hh.ru - его обязательно. sj.ru можно попробовать сделать вторым. Он находится в очень странном состоянии и возможна некорректная работа.!!!

*Делала только hh.ru*

In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
def parse_salary(ssalary):
    if ssalary == '':
        return ['', '', '']
    salary = ssalary.replace(' ', '')
    asalary = salary.split(' ')
    curr = asalary[-1]

    salary_from = ''
    salary_to = ''

    regex_num = re.compile('\d+')
    salary_nums = regex_num.findall(salary)

    pattern_from = r'от.+[\d]+.+'
    pattern_to = r'до.+[\d]+.+'
    pattern_both = r'[\d]+.[\d]+.+'

    if re.match(pattern_from, salary):
        salary_from = salary_nums[0]
    elif re.match(pattern_to, salary):
        salary_to = salary_nums[0]
    elif re.match(pattern_both, salary):
        salary_from = salary_nums[0]
        salary_to = salary_nums[1]
    return [salary_from, salary_to, curr]

In [4]:
user_query = str(input('Укажите запрос: '))

Укажите запрос:  Data scientist


In [5]:
# Без проверки на вводимые данные - предполагаем адекватность пользователя
user_limits = int(input('Укажите количество страниц для анализа(-1, если все): '))

Укажите количество страниц для анализа(-1, если все):  5


In [6]:
page = 0
main_link = 'https://hh.ru'

headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/83.0.4103.61 Safari/537.36'}
params = {'L_is_autosearch': 'false',
              'clusters': 'true',
              'enable_snippets': 'true',
              'text': user_query,
              'showClusters': 'true',
              'page': page}
vacancies = []

In [7]:
while True:
    if user_limits >= 0 and params['page'] + 1 > user_limits:  # остановка, если проанализировали необходимое количество 
        break
        
    response = requests.get(main_link + '/search/vacancy/', params=params, headers=headers)
    # print(f'Код ответа сервера: {response.status_code}')

    soup = bs(response.text, 'lxml')

    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})  # нашли блок с вакансиями
    
    if not vacancy_block:  # остановка, если не нашли блок с вакансиями
        break
    
    vacancy_list = vacancy_block.findChildren(recursive=False)

    if not vacancy_list:  # остановка, если список вакансий в блоке пуст
        break

    for vacancy in vacancy_list:
        vacancy_data = {}
        
        # наименование вакансии и ссылка на саму вакансию
        vacancy_link_a = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})
        try:
            vacancy_name = vacancy_link_a.getText()
            vacancy_link = str(vacancy_link_a['href'])
        except TypeError:
            # vacancy_name = ''
            # vacancy_link = ''
            continue
        except AttributeError:
            # vacancy_name = ''
            # vacancy_link = ''
            continue
            
        # сайт, откуда собрана вакансия
        vacancy_source = main_link
        
        # работодатель
        vacancy_employer_a = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'})
        try:
            vacancy_employer = vacancy_employer_a.getText()
        except AttributeError:
            vacancy_employer = ''
            
        # расположение
        vacancy_location_span = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'})
        try:
            vacancy_location = vacancy_location_span.getText()
        except AttributeError:
            vacancy_location = ''
            
        # предлагаемая зарплата
        vacancy_salary_div = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'})
        try:
            vacancy_salary_span = vacancy_salary_div.findChildren(recursive=False)
            if vacancy_salary_span:
                try:
                    vacancy_salary = vacancy_salary_span[0].getText()
                except AttributeError:
                    vacancy_salary = ''
            else:
                vacancy_salary = ''
        except AttributeError:
            vacancy_salary = ''
    
        vacancy_salary_array = parse_salary(vacancy_salary)
        vacancy_salary_from = vacancy_salary_array[0]
        vacancy_salary_to = vacancy_salary_array[1]
        vacancy_salary_curr = vacancy_salary_array[2]

        vacancy_data['name'] = vacancy_name
        vacancy_data['url'] = vacancy_link
        vacancy_data['source'] = vacancy_source
        vacancy_data['location'] = vacancy_location
        vacancy_data['employer'] = vacancy_employer
        vacancy_data['min_salary'] = vacancy_salary_from
        vacancy_data['max_salary'] = vacancy_salary_to
        vacancy_data['currency'] = vacancy_salary_curr

        vacancies.append(vacancy_data)

    params['page'] += 1

In [8]:
result = pd.DataFrame(vacancies)
result

,name,url,source,location,employer,min_salary,max_salary,currency
0,Data Scientist,https://hh.ru/vacancy/37369353?query=Data%20sc...,https://hh.ru,"Санкт-Петербург, Выборгская",ЦРТ | Группа компаний,,,
1,Pricing development manager,https://hh.ru/vacancy/37331973?query=Data%20sc...,https://hh.ru,Москва,British American Tobacco Russia,,,
2,Python Data Scientist,https://hh.ru/vacancy/36916080?query=Data%20sc...,https://hh.ru,"Москва, Площадь Революции и еще 3",Web Interactive World,150000,,руб.
3,Senior Data Scientist,https://hh.ru/vacancy/37294434?query=Data%20sc...,https://hh.ru,"Москва, Калужская",Gradient,300000,,руб.
4,Data Scientist (Limassol),https://hh.ru/vacancy/37362533?query=Data%20sc...,https://hh.ru,Москва,Easybrain,,,
5,Data Scientist,https://hh.ru/vacancy/37054908?query=Data%20sc...,https://hh.ru,"Москва, Марксистская и еще 2",ООО Варити,130000,,руб.
6,Data scientist,https://hh.ru/vacancy/37329687?query=Data%20sc...,https://hh.ru,Москва,SUNLIGHT,,150000,руб.
7,Data Scientist,https://hh.ru/vacancy/37003152?query=Data%20sc...,https://hh.ru,Санкт-Петербург,ООО Цифровое проектирование,110000,,руб.
8,Middle/Senior data scientist (fraud prevention),https://hh.ru/vacancy/36991107?query=Data%20sc...,https://hh.ru,Москва,ООО КИБЕРТОНИКА,140000,220000,руб.
9,Аналитик - эконометрист / Data scientist (Juni...,https://hh.ru/vacancy/37172711?query=Data%20sc...,https://hh.ru,Москва,ООО БСТ Менеджмент-Консалтинг,,,
